# Import packages

In [ ]:
import gym
import time
import numpy as np 

# Agent

In [ ]:
class QLearningAgent(object):
    def __init__(self, obs_n, act_n, learning_rate=0.01, gamma=0.9, e_greed=0.1):
        self.act_n = act_n      # number of actions
        self.lr = learning_rate # learning rate
        self.gamma = gamma      # decay parameter (reward)
        self.epsilon = e_greed  # epsilon-greedy
        self.Q = np.zeros((obs_n, act_n))

    # sample a action using epsilon-greedy
    def sample(self, obs):
        if np.random.uniform(0, 1) < (1.0 - self.epsilon): 
            action = self.predict(obs)
        else:
            action = np.random.choice(self.act_n) 
        return action

    # choosing an action from current state using Q(s,a) table
    def predict(self, obs):
        Q_list = self.Q[obs, :]
        maxQ = np.max(Q_list)
        action_list = np.where(Q_list == maxQ)[0]  # maxQ may have duplications
        action = np.random.choice(action_list)
        return action

    # Updating Q(s,a) table
    def learn(self, obs, action, reward, next_obs, done):
        """ on-policy
            obs: s_t
            action: a_t
            reward: r
            next_obs: s_t+1
            done: whether an episode is over
        """
        predict_Q = self.Q[obs, action]
        if done:
            target_Q = reward # fall into absorbing state
        else:
            target_Q = reward + self.gamma * np.max(self.Q[next_obs, :]) # Q-learning
        self.Q[obs, action] += self.lr * (target_Q - predict_Q) # # update table

    # # save Q(s,a) table locally
    def save(self):
        npy_file = './q_table.npy'
        np.save(npy_file, self.Q)
        print(npy_file + ' saved.')
    
    # load Q(s,a) table
    def restore(self, npy_file='./q_table.npy'):
        self.Q = np.load(npy_file)
        print(npy_file + ' loaded.')

# Training and test

In [ ]:
def run_episode(env, agent, render=False):
    total_steps = 0 # The number of steps in an episode
    total_reward = 0

    obs = env.reset() # start a new episode

    while True:
        action = agent.sample(obs) 
        next_obs, reward, done, _ = env.step(action) 
        # training using Q-Learning
        agent.learn(obs, action, reward, next_obs, done)

        obs = next_obs  
        total_reward += reward
        total_steps += 1 
        if render:
            env.render() 
        if done:
            break
    return total_reward, total_steps

def test_episode(env, agent):
    total_reward = 0
    obs = env.reset()
    while True:
        action = agent.predict(obs) # greedy
        next_obs, reward, done, _ = env.step(action)
        total_reward += reward
        obs = next_obs
        # time.sleep(0.5)
        # env.render()
        if done:
            break
    return total_reward

# Create environment and agent then start training

In [ ]:
env = gym.make("CliffWalking-v0")  # 0 up, 1 right, 2 down, 3 left

agent = QLearningAgent(
    obs_n=env.observation_space.n,
    act_n=env.action_space.n,
    learning_rate=0.1,
    gamma=0.9,
    e_greed=0.1)

# 500 episodes
for episode in range(500):
    ep_reward, ep_steps = run_episode(env, agent, False)
    print(f'Episode {episode}: steps = {ep_steps} , reward = {ep_reward:.1f}')

# Evaluation after training
test_reward = test_episode(env, agent)
print(f'test reward = {test_reward:.1f}')